<a href="https://colab.research.google.com/github/andres9119/Introduccion/blob/main/Evento_300.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime

# 1. Cargar el archivo Excel
archivo = input("Ingrese el nombre del archivo Excel (ej: datos2.xlsx): ").strip()
df = pd.read_excel(archivo)
descargado = df

# 2. Identificar columnas de tipo fecha
columnas_fecha = [col for col in df.columns if df[col].dtype == 'datetime64[ns]']

# 3. Formatear columnas de fecha como texto en formato dd/mm/yyyy
for col in columnas_fecha:
    df[col] = df[col].dt.strftime('%d/%m/%Y')

# 4. Filtrar registros de ajustes
valores_ajuste = ['R', '6', 'D']
ajustes = df[df['ajuste_'].isin(valores_ajuste)]
sin6dr = df[~df['ajuste_'].isin(valores_ajuste)]

# 5. En el sin6dr, separar Norte de Santander y otros departamentos
norte_santander = sin6dr[sin6dr['ndep_proce'].str.upper() == 'NORTE SANTANDER']
otros_departamentos = sin6dr[sin6dr['ndep_proce'].str.upper() != 'NORTE SANTANDER']

Ingrese el nombre del archivo Excel (ej: datos2.xlsx): 300_junio_2025.xlsx


In [ ]:
df_norte = norte_santander

# 2. Pedir el año a procesar
año_procesar = int(input("Ingrese el año que desea procesar (ej: 2025): "))

# 3. Filtrar por año
df_norte = df_norte[df_norte['año'] == año_procesar]

# 4. Convertir 'fec_not' a tipo fecha
df_norte['fec_not'] = pd.to_datetime(df_norte['fec_not'], format='%d/%m/%Y', errors='coerce')

Ingrese el año que desea procesar (ej: 2025): 2025


In [ ]:
# 5. Inicializar tablas de salida
depurados = []
duplicados = []

# 6. Agrupar por documento en df_norte (ya filtrado)
for num_ide, grupo in df_norte.groupby('num_ide_'):
    if len(grupo) == 1:
        depurados.append(grupo.iloc[0])
    else:
        # Comparar fechas entre todos los pares
        grupo = grupo.sort_values('fec_not')
        indices = list(grupo.index)
        ya_procesados = set()

        for i in range(len(indices)):
            if indices[i] in ya_procesados:
                continue
            candidato = grupo.loc[indices[i]]
            candidatos_duplicados = []

            for j in range(i+1, len(indices)):
                if indices[j] in ya_procesados:
                    continue
                fecha_i = candidato['fec_not']
                fecha_j = grupo.loc[indices[j], 'fec_not']

                if pd.notnull(fecha_i) and pd.notnull(fecha_j):
                    if abs((fecha_i - fecha_j).days) <= 90:
                        candidatos_duplicados.append(grupo.loc[indices[j]])
                        ya_procesados.add(indices[j])

            if candidatos_duplicados:
                candidatos_duplicados.append(candidato)
                candidatos_df = pd.DataFrame(candidatos_duplicados)

                # Aplicar criterios A, B, C, D
                candidatos_df['apl_sa'] = candidatos_df['apl_sa'].fillna(0)
                candidatos_df['apl_vac'] = candidatos_df['apl_vac'].fillna(0)
                candidatos_df['pac_hos_'] = candidatos_df['pac_hos_'].fillna(0)
                prioridad_upgd = {'ese': 1, 'centro de salud': 2, 'ips': 3}
                candidatos_df['prioridad_upgd'] = candidatos_df['nom_upgd'].str.lower().map(prioridad_upgd).fillna(99)
                candidatos_df = candidatos_df.sort_values(
                    by=['apl_sa', 'apl_vac', 'pac_hos_', 'prioridad_upgd', 'semana'],
                    ascending=[False, False, False, True, True]
                )

                ganador = candidatos_df.iloc[0]
                depurados.append(ganador)

                duplicados.extend(candidatos_df.iloc[1:].to_dict(orient='records'))
            else:
                depurados.append(candidato)

# 7. Convertir resultados a DataFrame
depurados_df = pd.DataFrame(depurados)
duplicados_df = pd.DataFrame(duplicados)

# 8. Formatear fechas para que se guarden como dd/mm/yyyy
columnas_fecha_depurados = [col for col in depurados_df.columns if depurados_df[col].dtype == 'datetime64[ns]']
columnas_fecha_duplicados = [col for col in duplicados_df.columns if duplicados_df[col].dtype == 'datetime64[ns]']

for col in columnas_fecha_depurados:
    depurados_df[col] = depurados_df[col].dt.strftime('%d/%m/%Y')

for col in columnas_fecha_duplicados:
    duplicados_df[col] = duplicados_df[col].dt.strftime('%d/%m/%Y')

# 9. Guardar todo en el mismo archivo Excel
with pd.ExcelWriter('resultado_ajustes_final_2024.xlsx', engine='openpyxl') as writer:
    descargado.to_excel(writer, sheet_name='Descargado', index=False)
    ajustes.to_excel(writer, sheet_name='Ajustes', index=False)
    sin6dr.to_excel(writer, sheet_name='Sin6dr', index=False)
    norte_santander.to_excel(writer, sheet_name='Norte_Santander_Original', index=False)
    otros_departamentos.to_excel(writer, sheet_name='Otros_Departamentos', index=False)
    depurados_df.to_excel(writer, sheet_name='Depurados', index=False)
    duplicados_df.to_excel(writer, sheet_name='Duplicados', index=False)

print("✅ Proceso completamente finalizado. Archivo 'resultado_ajustes_final.xlsx' generado con todas las hojas.")

✅ Proceso completamente finalizado. Archivo 'resultado_ajustes_final.xlsx' generado con todas las hojas.
